## Code to draw a map of the local authoritys and colour them according to an average rating that they issue

In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import numpy
import plotly.express as px
import pandas as pd

df = pd.read_csv("FoodRating.csv")

LocalAuthority = []
Score = []
Lat = []
Long = []

# create a pivot table to agregate the RatingValue
df_pt = df.pivot_table(index='LocalAuthorityName', columns='RatingValue', aggfunc = {'RatingValue':len}, fill_value=0)

for index, row in df_pt.iterrows():

    geolocator = Nominatim(user_agent="NJB")
    location = geolocator.geocode(f"{index}, UK")
    
    if location != None:
        #print (location)
        score = (row[0] * 5) + (row[1] * 4) + (row[2] * 3) + (row[3] * 4) + (row[4] * 5)
        num = row[0] + row[1] + row[2] + row[3] + row[4]
                
        LocalAuthority.append (index)
        Score.append (score/num)
        if location.latitude != None:
            Lat.append (location.latitude)
        else:
            Lat.append (0)
        if location.longitude != None:
            Long.append (location.longitude)
        else:
            Long.append(0)
   



df_map = pd.DataFrame(list(zip(LocalAuthority, Score, Lat, Long)))
df_map.columns = ['LocalAuthorityName', 'Score', 'lat','long']

color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(df_map, 
                        lat="lat",
                        lon="long", 
                        hover_name="LocalAuthorityName", 
                        hover_data=["Score"],
                        color="Score",
                        color_continuous_scale="RdYlGn",
                        
                        zoom=8, 
                        height=800,
                        width=800)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_traces(marker={'size': 15})
fig.show()



## another way of drawing the map

In [ ]:
from urllib.request import urlopen
import json
import pandas as pd
import plotly.express as px


with urlopen('https://raw.githubusercontent.com/thomasvalentine/Choropleth/main/Local_Authority_Districts_(December_2021)_GB_BFC.json') as response:
    Local_authorities = json.load(response)


la_data = []
# Iterative over JSON
for i in range(len(Local_authorities["features"])):
    # Extract local authority name
    la = Local_authorities["features"][i]['properties']['LAD21NM']
    # Assign the local authority name to a new 'id' property for later linking to dataframe
    Local_authorities["features"][i]['id'] = la
    # While I'm at it, append local authority name to a list to make some dummy data to test, along with i for a value to test on map
    la_data.append([la,i])



import pandas as pd

# turn dummy data into a dataframe
df2 = pd.DataFrame(la_data)
# update column names
df2.columns = ['LocalAuthorityName','Val']

df_merge = pd.merge(df_map, df2, how = "inner", on="LocalAuthorityName")


fig = px.choropleth_mapbox(df_merge,
                           geojson=Local_authorities,
                           locations='LocalAuthorityName',
                           color='Score',
                           featureidkey="properties.LAD21NM",
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           center={"lat": 55.09621, "lon": -4.0286298},
                           zoom=6,
                            height=800,
                            width=800,
                           labels={'val':'value'})

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## and another map, this time with the raw data

In [ ]:
import plotly.express as px
import pandas as pd

df = pd.read_csv("FoodRating.csv")

color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(df, 
                        lat="Geocode_Lat",
                        lon="Geocode_Long", 
                        hover_name="Business Name", 
                        hover_data=["Business Type", "RatingValue"],
                        color="RatingValue",
                        color_continuous_scale="RdYlGn",
                        
                        zoom=8, 
                        height=800,
                        width=800)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_traces(marker={'size': 7})
fig.show()